# 1D bedrock channel erosion using the Stream-Power model

The Python/Numpy code below has been adapted from a Matlab code originally written by K.X. Whipple (2010).

## Goal

- Real situation: we have some existing code (Python/Numpy/Matplotlib) and we want to adapt it to create an interactive, nice-looking dashboard.
- We'll use `ipywidgets` and `ipympl` 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib widget

In [ ]:
# This code has been adapted from the Matlab script
# "detach_ftfs.m" originally written by K.X. Whipple.

def log_slope(z, dx):
    """
    Compute log slope from elevations.
    """
    return np.log10((z[0:-1] - z[1:]) / dx)


def steady_profile(x, U, K, inputs):
    """
    Compute the steady-state channel profile
    (elevations) given U and K.
    """
    x_temp = inputs['x']**(1 - (inputs['h'] * inputs['m_n']))
    z_coef = (U / K)**(1. / inputs['n'])
    z_coef *= inputs['ka']**(-inputs['m_n']) * (1. - inputs['h'] * inputs['m_n'])**-1
    z = ((-1.0 * x_temp) + inputs['L']**(1.0 - inputs['h'] * inputs['m_n'])) * z_coef
    return z


def setup_model(ka=6.69, h=1.67, m_n=0.5, n=1.0, U=0.004, K=5e-5,
                z_init='steady', islope=0.01, plat_z=1.0, Ui=0.002, Ki=5e-4):
    """Set model inputs.
    
    Parameters
    ----------
    ka : float
        Hack's law coefficient
    h : float
        Hack's law exponent
    m_n : float
        SPL m/n ratio
    n : float
        SPL slope exponent
    U : float
        Uplift rate [m yr^-1]
    K : float
        SPL coefficient (bedrock erodibility)
    z_init : {'ramp', 'flat', 'steady'}
        Initial channel profile
    islope : float
        Initial channel slope for z_init = 'ramp'
    plat_z : float
        Initial plateau elevation for z_init = 'flat'
    Ui : float
        Initial uplift rate for z_init = 'steady'
    Ki : float
        Initial bedrock erodibility for z_init = 'steady'
    """
    inputs = {}
    state = {}
    
    # channel length [m]
    inputs['L'] = L = 30000
    # channel initialization distance [m]
    x_crit = 300
    
    n_nodes = 100
    dx = 1.0 * L / n_nodes
    n_nodes = int(100 - round(x_crit / dx) + 1)
    inputs['x'] = x = np.linspace(x_crit, L, n_nodes)
    inputs['dx'] = 1.0 * L / n_nodes
    
    inputs['ka'] = ka
    inputs['h'] = h
    inputs['m_n'] = m_n
    inputs['n'] = n
    inputs['m'] = m = m_n * n
    inputs['U'] = U
    inputs['K'] = K
    
    if z_init == 'ramp':
        z = (L - x) * islope
    elif z_init == 'flat':
        z = np.ones_like(x) * plat_z
        z[-1] = 0
    elif z_init == 'steady':
        z = steady_profile(x, Ui, Ki, inputs)
    else:
        raise ValueError("unknown initial conditions '{}'"
                     .format(z_init))
    state['z'] = z
    
    # simulation duration [yr]
    inputs['T'] = 100000
    # stability parameter (CFL)
    inputs['stabil'] = stabil = 1
    # time step
    inputs['dt'] = dt = dx / (K * (ka**m) * (L**(h*m))) / stabil 
    
    # some constants in the equation to be solved 
    # outside of the time loop to maximize efficiency.
    inputs['dU'] = U * dt
    inputs['Kx'] = (K * (ka**m) * dt / ((dx)**n)) * (x**(h*m))
    inputs['area'] = area = ka * x**h
    
    # compute log(slope) and log(area)
    state['log_s'] = log_slope(z, dx)
    inputs['log_a'] = np.log10((area[0:-1] + area[1:]) / 2.)
    
    return inputs, state
    
    
def run_step(inputs, state):
    """Run one single step."""

    z = state['z']
    z1 = state['z'].copy()
    
    # calculate channel profile after one time step
    z1[:-1] = z[:-1] + inputs['dU']
    z1[:-1] -= inputs['Kx'][:-1] * np.abs(z[1:] - z[:-1])**inputs['n']
    
    # stability => no increase in elevation
    # along the profile
    z1[:-1] = np.maximum(z1[:-1], z[1:])
 
    # boundary conditions
    z1[-1] = z[-1]

    # update z
    state['z'] = z1.copy()
    
    # log(slope)
    state['log_s'] = log_slope(z1, inputs['dx'])
    

def run(n_snapshots=5, **params):
    """Run one simulation."""

    inputs, state = setup_model(**params)

    # init time increments and outputs
    t = 0.
    save_inc = 0
    save_max_inc = int((inputs['T'] / inputs['dt']) / n_snapshots)
    
    outputs = {
        't': [t],
        'z': [state['z']],
        'log_s': [state['log_s']]
    }

    # Main time loop
    while t < inputs['T'] + inputs['dt']:

        run_step(inputs, state)

        # save intermediate solutions
        if save_inc > save_max_inc:
            outputs['t'].append(t)
            outputs['z'].append(state['z'].copy())
            outputs['log_s'].append(state['log_s'].copy())
            save_inc = 0

        # increment time step
        t += inputs['dt']
        save_inc += 1
    
    return inputs, outputs


def plot(inputs, outputs):
    """Run one simulation and plot the outputs."""

    plt.ioff()

    fig, ax = plt.subplots(nrows=2, figsize=(6, 6))

    # plot initial condition (in red)
    ax[0].plot(inputs['x'], outputs['z'][0], c='r')
    ax[1].plot(inputs['log_a'], outputs['log_s'][0], c='r')

    # plot intemediate solutions
    for i in range(1, len(outputs['t']) - 1):
        clr_index = int(255. * i / len(outputs['t']))
        clr = plt.cm.winter(clr_index)
        ax[0].plot(inputs['x'], outputs['z'][i], c=clr)
        ax[1].plot(inputs['log_a'], outputs['log_s'][i], c=clr)
   
    # plot the final solution (in green)
    ax[0].plot(inputs['x'], outputs['z'][-1], c='g')
    ax[1].plot(inputs['log_a'], outputs['log_s'][-1], c='g')

    # calculate and plot the analytic solution at steady-state
    z_steady = outputs['z'][-1][-1] + steady_profile(inputs['x'], inputs['U'], inputs['K'], inputs)
    log_s = log_slope(z_steady, inputs['dx'])
    ax[1].plot(inputs['log_a'], log_s, c='k', alpha=0.7, ls='--')
    ax[0].plot(inputs['x'], z_steady, c='k', alpha=0.7, ls='--')

    # plot refinements
    plt.setp(ax[0],
             xlabel='distance from divide [m]',
             ylabel='elevation [m]')
    plt.setp(ax[1],
             xlabel='log drainage area [$\log$(m$^2$)]',
             ylabel='log channel gradient [#]')
    plt.tight_layout()
    
    return fig

In [ ]:
inputs, outputs = run()
fig = plot(inputs, outputs)

fig.canvas

---

## Exercise 1

Create a "Run model" button that will run a simulation and (re)generate the figure above.

Hint: show the figure in a `ipywidgets.Ouput` widget with `IPython.display.display`.

## Exercise 2

In addition to the "Run model" button, create two input widgets that allow setting values for `K` (SPL coefficient) and `U` (uplift rate) in the simulation.

Hint: you can use a variety of widgets like `ipywidgets.FloatText`, `ipywidgets.FloatSlider` and `ipywidgets.FloatLogSlider`

**Optional**

- Instead of using a button, re-generate the figure when interacting with input widgets

## Exercise 3

Add widgets for more inputs, try organize them using better layouts (maybe group parameters into tabs or accordions UI elements), and then render the results as a standalone dashboard using `Voila`.

## Exercise 4

Let's try leveraging `ipympl` to create a more interactive and responsive figure!

Hints:

- instead of re-creating the matplotlib figure from scratch, start creating a figure from a model run with some default parameter values and show it directly (you can copy code from the `plot` function above). Let's skip drawing the intermediate river profiles for more simplicity.
- create a widget "callback" function that will re-run a new simulation (possibly with new parameter values) and that will update the line data (x, y) in the existing plot.
